In [1]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MultipleLocator
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import Common_Functions as cmn

In [2]:
sns.set_context("paper")
sns.set_style("ticks")

In [ ]:
rf_prediction =
gbr_prediction =
OLS_prediction =
XBG_prediction =
observed_price =

In [ ]:
fig, ax = plt.subplots()
plt.plot(rf_prediction, alpha=0.8, label=r"Random Forest Prediction")
plt.plot(gbr_prediction, alpha=0.8, label=r"Gradient Boosted Regressor Prediction")
plt.plot(OLS_prediction, alpha=0.8, label=r"OLS Prediction")
plt.plot(XBG_prediction, alpha=0.8, label=r"XBG Regressor Prediction")

plt.plot(observed_price, alpha=0.8, label=r"Obeserved Price")

# always label your axes
plt.xlabel(r"Sample Number")
plt.ylabel(r"House Price")

# create a legend
plt.legend(loc="upper left")
plt.xlim(0, 40)
plt.ylim(0, 500000)
plt.tight_layout()

plt.show()